In [8]:
import pandas as pd
import torch
import torch.nn as nn
import numpy as np
import torch.optim as optim
from torch.optim import Adam

ModuleNotFoundError: No module named 'torch'

In [35]:
reviews=''

In [4]:
with open('sentiment.txt','r') as f:
     reviews=f.read()

In [5]:
 data=pd.DataFrame([rv.split('\t') for rv in reviews.split('\n')])

In [6]:
data.columns=['reviews','sentiment']

In [7]:
data.head()

,reviews,sentiment
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1


In [40]:
class vocobalary_builder:
      word_to_index={}
      index_to_word={}
      counter=0
      max_len=0

      def add_token(self,token):
          if token not in self.word_to_index:
             self.word_to_index[token]=self.counter
             self.index_to_word[self.counter]=token
             self.counter+=1

      def add_tokens(self,review):
          review=review.lower()
          split=review.split()
          if len(split)>self.max_len:
              self.max_len=len(split)
          for token in review.split():
              self.add_token(token)

      def get_index_of_token(self,token):
          return self.word_to_index.get(token)

      def get_token_of_index(self,index):
          return self.index_to_word.get(index)

      def get_pedded_review(self,review):
          review=review.lower()
          split=review.split()
          #print(split)
          return [self.word_to_index.get(token) for token in split] + [0]*(self.max_len-len(split))

          # do backward padding




In [41]:
vb=vocobalary_builder()
encoded_sentences=[]
for index, row in data.iterrows():
    vb.add_tokens(row['reviews'])



for index, row in data.iterrows():
    encode_sent=vb.get_pedded_review(row['reviews'])
    encoded_sentences.append(encode_sent)




In [42]:
vb.get_index_of_token('the')

18

In [43]:
batch_size=1

In [44]:
print(vb.get_pedded_review('the movie was good'))

[18, 5, 14, 91, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [45]:
'''
model design
i will keep first layer as embedding layer
second layer will be lstm layer
third layer will be sigmoid layer
'''



class sentiment_analyzer(nn.Module):
      def __init__(self,vocab_dim,embedding_dim,n_hidden,n_output,no_of_layers,drop_p=0.8):
          super().__init__()
          self.vocab_dim=vocab_dim
          self.embedding_dim=embedding_dim
          self.no_of_layers=no_of_layers
          self.n_hidden=n_hidden
          self.n_output=n_output
          self.embedding=nn.Embedding(vocab_dim,embedding_dim)
          self.lstm=nn.LSTM(embedding_dim,n_hidden,no_of_layers,batch_first=True,dropout=0.8)
          self.dropout=nn.Dropout(drop_p)
          self.fc = nn.Linear(n_hidden, n_output)

          self.sigmoid=nn.Sigmoid()


      def forward(self,input_words):
          # all input words become part of our embedding layer during training
          embedded_words=self.embedding(input_words)
          lstm_out,h=self.lstm(embedded_words)
          lstm_out =self.dropout(lstm_out)
          lstm_out = lstm_out.contiguous().view(-1,self.n_hidden)
          fc_out=self.fc(lstm_out)
          sigmoid_out=self.sigmoid(fc_out)
          sigmoid_out = sigmoid_out.view(batch_size, -1)
          sigmoid_last=sigmoid_out[:, -1]
          return sigmoid_last,h



In [46]:
#convert encoded_sentences to numpy array
encoded_sentences=np.array(encoded_sentences)

In [47]:
vocab_len=len(vb.word_to_index)
embedding_dim=80
n_hidden=2
n_output=1
no_of_layers=2
model=sentiment_analyzer(vocab_len,embedding_dim,n_hidden,n_output,no_of_layers)
#optimizer=optim.Adam(model.parameters(),lr=0.001)
optimizer = Adam(model.parameters(), lr=0.001)

loss_fun=nn.BCELoss()


In [48]:
labels = np.array([int(x) for x in data['sentiment'].values])



In [49]:
#now lets split train test and validation set
from sklearn.model_selection import train_test_split
train_x,test_x,train_y,test_y=train_test_split(encoded_sentences,labels,test_size=0.20)
test_x,val_x,test_y,val_y=train_test_split(encoded_sentences,labels,test_size=0.5)


In [50]:
train_x = torch.Tensor(train_x).long()
valid_x = torch.Tensor(val_x).long()
test_x = torch.Tensor(test_x).long()

train_y = torch.Tensor(train_y)
valid_y = torch.Tensor(val_y)
test_y = torch.Tensor(test_y)

In [51]:
from torch.utils.data import TensorDataset,DataLoader

In [52]:
train_data = TensorDataset(train_x, train_y)
valid_data = TensorDataset(valid_x, valid_y)
test_data = TensorDataset(test_x, test_y)

In [53]:
#now we will create tensor dataset and decleare batch size =1
batch_siz=1
train_loader=DataLoader(train_data,batch_size=batch_siz,shuffle=True)
valid_loader=DataLoader(valid_data,batch_size=batch_siz,shuffle=True)
test_loader=DataLoader(test_data,batch_size=batch_siz,shuffle=True)


In [54]:
print_every=500
step=0
n_epochs=3

In [55]:
#now we will iterate trainLoader to train model

for epoch in range(3):
    for inputs,labels in train_loader:
        step+=1
        model.zero_grad()
        pred_label,h=model(inputs)
        loss=loss_fun(pred_label,labels.float())
        loss.backward()
        nn.utils.clip_grad_norm(model.parameters(), 5)
        optimizer.step()
        #print(loss)
        if (step % print_every) == 0:
            model.eval()
            valid_losses = []
            for v_inputs, v_labels in valid_loader:

                v_output, v_h = model(v_inputs)
                v_loss = loss_fun(v_output, v_labels.float())
                valid_losses.append(v_loss.item())
            print("Epoch: {}/{}".format((epoch+1), n_epochs),
                  "Step: {}".format(step),
                  "Training Loss: {:.4f}".format(loss.item()),
                  "Validation Loss: {:.4f}".format(np. mean(valid_losses)))
            model.train()


<ipython-input-55-004f00678d9a>:10: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
  nn.utils.clip_grad_norm(model.parameters(), 5)


Epoch: 1/3 Step: 500 Training Loss: 0.7624 Validation Loss: 0.7056
Epoch: 1/3 Step: 1000 Training Loss: 0.4867 Validation Loss: 0.7009
Epoch: 1/3 Step: 1500 Training Loss: 0.5161 Validation Loss: 0.6964
Epoch: 1/3 Step: 2000 Training Loss: 0.5281 Validation Loss: 0.6961
Epoch: 2/3 Step: 2500 Training Loss: 0.5178 Validation Loss: 0.6960
Epoch: 2/3 Step: 3000 Training Loss: 0.6635 Validation Loss: 0.6943
Epoch: 2/3 Step: 3500 Training Loss: 0.5808 Validation Loss: 0.6938
Epoch: 2/3 Step: 4000 Training Loss: 0.5824 Validation Loss: 0.6939
Epoch: 2/3 Step: 4500 Training Loss: 0.8052 Validation Loss: 0.6932
Epoch: 3/3 Step: 5000 Training Loss: 0.8064 Validation Loss: 0.6934
Epoch: 3/3 Step: 5500 Training Loss: 0.7116 Validation Loss: 0.6933
Epoch: 3/3 Step: 6000 Training Loss: 0.6058 Validation Loss: 0.6934
Epoch: 3/3 Step: 6500 Training Loss: 0.7774 Validation Loss: 0.6934
Epoch: 3/3 Step: 7000 Training Loss: 0.6343 Validation Loss: 0.6930


In [56]:
torch.save(model.state_dict(), 'lstm_sentiment_analyzer.pkl')



In [57]:
model.eval()
test_losses=[]
num_correct=0
for inputs,label in test_loader:
    output,h=model(inputs)
    loss=loss_fun(output,label.float())
    #print(output)
    preds = torch.round(output.squeeze())
    test_losses.append(loss.item())
    correct_tensor = preds.eq(label.float().view_as(preds))
    correct = np.squeeze(correct_tensor.numpy())
    #print('correct value',correct)
    num_correct += np.sum(correct)


print("Test Loss: {:.4f}".format(np.mean(test_losses)))
print("Test Accuracy: {:.2f}".format(num_correct/len(test_loader.dataset)))

Test Loss: 0.6933
Test Accuracy: 0.49


In [58]:
def predict_review(review):
    model.eval()
    review=vb.get_pedded_review(review)
    review_tensor=torch.tensor(review)
    output,h=model(review_tensor)
    print(output)
    msg = "This is a positive review." if output >= 0.5 else           "This is a negative review."
    print(msg)
    print('Prediction = ' + str(output))

In [62]:
predict_review('worst movie')


tensor([0.4960], grad_fn=<SelectBackward0>)
This is a negative review.
Prediction = tensor([0.4960], grad_fn=<SelectBackward0>)
